In [ ]:
import csv
import pandas as pd
import seaborn as sns


# Caminho para o arquivo CSV
file_path = "user_courses_review_09_2023.csv"

# Lista para armazenar as linhas de dados
data_list = []

# Numero de colunas esperado por cada linha
EXPECTED_COLUMNS = 4  # Troque esse numero conforme necessário

# Abrir o arquivo CSV e ler linha por linha
with open(file_path, "r", encoding="utf-8", newline="") as file:
    reader = csv.reader(file)

    # Opcional: Ler o cabeçalho se existir
    header = next(reader)

    # Ler cada linha
    for row in reader:
        # Verificar se a linha tem o número esperado de colunas
        if len(row) == EXPECTED_COLUMNS:
            data_list.append(row)
        else:
            print(f"Linha ignorada: {reader.line_num}: {row}")

# Converter a lista de dados em um DataFrame do Pandas
df = pd.DataFrame(data_list, columns=header)

# Mostrar as primeiras linhas do DataFrame
print(df.head())

In [ ]:
# Checando os data types das colunas
data_types = df.dtypes

print(data_types)

In [ ]:
# Converter 'review_rating' para numérico
if df["review_rating"].dtype == "object":
    # Força a a conversão para numérico, tratando erros como NaN
    df["review_rating"] = pd.to_numeric(df["review_rating"], errors="coerce")

In [ ]:
# Verificar se a conversão foi bem-sucedida e valores ausentes
new_data_types = df.dtypes
missing_values = df.isnull().sum()

print(new_data_types)
print(missing_values)

In [ ]:
import matplotlib.pyplot as plt

# Summary estatistica para review_rating
review_rating_summary = df["review_rating"].describe()
print(review_rating_summary)

In [ ]:
# Grafico de distribuição de review_rating
plt.figure(figsize=(10, 6))
plt.hist(
    df["review_rating"].dropna(),
    bins=range(1, 7),
    align="left",
    rwidth=0.9,
    color="skyblue",
    edgecolor="gray",
)
plt.title("Distribuição de Avaliações dos Cursos")
plt.xlabel("Avaliação")
plt.ylabel("Frequência")
plt.xticks(range(1, 6))
plt.grid(axis="y")
plt.show()

In [ ]:
# Remover linhas com dados ausentes
df_cleaned = df.dropna().copy()

In [ ]:
# Checkando o novo shape do DataFrame
print(
    f"Shape do DataFrame após remoção de linhas com dados ausentes: {df_cleaned.shape}"
)

In [ ]:
# Para analise de correlação, primeiro precisamos adicionar uma nova coluna com o tamanho dos comentarios
df_cleaned["comment_length"] = df_cleaned["review_comment"].apply(len)

In [ ]:
# Calcular a correlação entre review_rating e comment_length
correlation = df_cleaned[["review_rating", "comment_length"]].corr()

print(correlation)

In [ ]:
import re

# Define regex patterns para procurar palavras-chave
patterns = {
    "positive": re.compile(
        r"\b(good|great|excellent|amazing|love|enjoy|awesome|best|fantastic)\b",
        re.IGNORECASE,
    ),
    "negative": re.compile(
        r"\b(bad|terrible|awful|hate|dislike|poor|worst|disappoint)\b", re.IGNORECASE
    ),
    "question": re.compile(r"\?"),
    "exclamation": re.compile(r"\!"),
    "capitalized": re.compile(
        r"\b[A-Z]{2,}\b"
    ),  # palavras com 2 ou mais letras maiúsculas
}

# Inicializa um dicionário para armazenar as contagens para cada padrão
pattern_counts = {key: 0 for key in patterns}

# Scan cada comentário e contar as ocorrências de cada padrão
for comment in df_cleaned["review_comment"]:
    for key, pattern in patterns.items():
        if re.search(pattern, comment):
            pattern_counts[key] += 1

pattern_counts

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
import string


# Função para pré-processar os comentários
def preprocess_text(text):
    # Remove pontuação e converte para minúsculas
    text = text.translate(str.maketrans("", "", string.punctuation)).lower()
    return text


# Aplicar a função de pré-processamento
df_cleaned["review_comment"] = df_cleaned["review_comment"].apply(preprocess_text)

# Dividir os dados em conjunto de treino e teste
X = df_cleaned.loc[:, "review_comment"]
Y = df_cleaned.loc[:, "review_comment"]

# Dividir os dados entre treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [ ]:
# Inicializar a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words="english")

# Inicializar o modelo Naive Bayes Multinomial
model = MultinomialNB()

# Criar uma pipeline que combina o vetorizar e o modelo
pipeline = make_pipeline(vectorizer, model)

# Treinar o modelo
pipeline.fit(X_train, Y_train)

# Fazer previsões no conjunto de teste
Y_pred = pipeline.predict(X_test)

# Calcular e exibir o relatório de classificação
report = classification_report(Y_test, Y_pred)

# Exibir a matriz de confusão
conf_matrix = confusion_matrix(Y_test, Y_pred)
report, conf_matrix

In [ ]:
from sklearn.utils import resample

# Separar as classes minoritárias e majoritárias
df_majority = df_cleaned[df_cleaned["review_rating"] == 5]
df_minority = df_cleaned[df_cleaned["review_rating"] < 5]

# Upsample a classe minoritária
df_minority_upsampled = resample(
    df_minority,
    replace=True,  # Amostragem com reposição
    n_samples=len(
        df_majority
    ),  # Para igualar o número de amostras da classe majoritária
    random_state=42,
)  # Para reprodutibilidade

# Combinar a classe majoritária com a classe minoritária upsampled
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Verificar o novo balanceamento das classes
print(df_balanced["review_rating"].value_counts())

In [ ]:
# Separar os dados balanceados em X e Y
X_balanced = df_balanced["review_comment"]
Y_balanced = df_balanced["review_rating"]

# Dividir os dados balanceados entre treino e teste
X_train_balanced, X_test_balanced, Y_train_balanced, Y_test_balanced = train_test_split(
    X_balanced, Y_balanced, test_size=0.2, random_state=42
)

# Treinar o modelo novamente com os dados balanceados
pipeline_upsampled = make_pipeline(vectorizer, model)
pipeline_upsampled.fit(X_train_balanced, Y_train_balanced)

# Fazer previsões no conjunto de teste balanceado
Y_pred_balanced = pipeline_upsampled.predict(X_test_balanced)

# Calcular e exibir o relatório de classificação para os dados balanceados
report_balanced = classification_report(Y_test_balanced, Y_pred_balanced)

# Calcular a matriz de confusão para os dados balanceados
conf_matrix_balanced = confusion_matrix(Y_test_balanced, Y_pred_balanced)

report_balanced, conf_matrix_balanced

### Classification Task

In [ ]:
# Define a função para converter o 'rating' em 'good' ou 'bad'
def binary_rating(rating):
    if rating >= 4:
        return "good"
    else:
        return "bad"


# Aplica a função de conversão ao DataFrame na coluna 'review_rating'
binary_data = df_cleaned.copy()
binary_data["review_rating"] = binary_data["review_rating"].apply(binary_rating)

# Separar os dados binários em X e Y
X_binary = binary_data["review_comment"]
Y_binary = binary_data["review_rating"]

# Dividir os dados binários entre treino e teste
X_train_binary, X_test_binary, Y_train_binary, Y_test_binary = train_test_split(
    X_binary, Y_binary, test_size=0.2, random_state=42
)

# Treinar o modelo novamente com os dados binários
pipeline_binary = make_pipeline(vectorizer, model)
pipeline_binary.fit(X_train_binary, Y_train_binary)

# Fazer previsões no conjunto de teste binário
Y_pred_binary = pipeline_binary.predict(X_test_binary)

# Calcular e exibir o relatório de classificação para os dados binários
report_binary = classification_report(Y_test_binary, Y_pred_binary, output_dict=True)

# Calcular a matriz de confusão para os dados binários
conf_matrix_binary = confusion_matrix(Y_test_binary, Y_pred_binary)

# Cnoverte o reporte de classificação em um DataFrame
report_binary_df = pd.DataFrame(report_binary).transpose()

report_binary_df

In [ ]:
# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_binary,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["bad", "good"],
    yticklabels=["bad", "good"],
)
plt.title("Matriz de Confusão - Classificação Binária")
plt.xlabel("Previsão")
plt.ylabel("Real")
plt.show()

In [ ]:
# Separar os dados majoritários e minoritários
df_majority = binary_data[binary_data["review_rating"] == "good"]
df_minority = binary_data[binary_data["review_rating"] == "bad"]

# Oversample a classe minoritária
df_minority_upsampled_binary = resample(
    df_minority,
    replace=True,  # Amostragem com reposição
    n_samples=len(
        df_majority
    ),  # Para igualar o número de amostras da classe majoritária
    random_state=42,
)  # Para reprodutibilidade

# Combinar a classe majoritária com a classe minoritária upsampled
df_balanced_binary = pd.concat([df_majority, df_minority_upsampled_binary])

# Verificar o novo balanceamento das classes
print(df_balanced_binary["review_rating"].value_counts())

In [ ]:
# Separar os dados upsampled binary data em features(X) e labels(Y)
X_upsampled_binary = df_balanced_binary["review_comment"]
Y_upsampled_binary = df_balanced_binary["review_rating"]

# Dividir os dados balanceados entre treino e teste
(
    X_train_upsampled_binary,
    X_test_upsampled_binary,
    Y_train_upsampled_binary,
    Y_test_upsampled_binary,
) = train_test_split(
    X_upsampled_binary, Y_upsampled_binary, test_size=0.2, random_state=42
)

# Treinar o modelo novamente com os dados balanceados binários
pipeline_upsampled_binary = make_pipeline(vectorizer, model)
pipeline_upsampled_binary.fit(X_train_upsampled_binary, Y_train_upsampled_binary)

# Fazer previsões no conjunto de teste balanceado binário
Y_pred_upsampled_binary = pipeline_upsampled_binary.predict(X_test_upsampled_binary)

# Calcular e exibir o relatório de classificação para os dados balanceados binários
report_upsampled_binary = classification_report(
    Y_test_upsampled_binary, Y_pred_upsampled_binary, output_dict=True
)

# Calcular a matriz de confusão para os dados balanceados binários
conf_matrix_upsampled_binary = confusion_matrix(
    Y_test_upsampled_binary, Y_pred_upsampled_binary
)

# Converte o reporte de classificação em um DataFrame
report_upsampled_binary_df = pd.DataFrame(report_upsampled_binary).transpose()

report_upsampled_binary_df

In [ ]:
# Fazer o plot da matriz de confusão como heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_upsampled_binary,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["bad", "good"],
    yticklabels=["bad", "good"],
)
plt.title("Matriz de Confusão - Classificação Binária Balanceada")
plt.xlabel("Previsão")
plt.ylabel("Real")
plt.show()

### Teste Set

In [ ]:
# Carregar o arquivo de teste
test_file = pd.read_csv(
    "user_courses_review_test.csv",
    encoding="utf-8",
)

In [ ]:
# Remover linhas com dados ausentes em 'review_comment' ou 'review_rating'
test_set_clean = test_file.dropna(subset=["review_comment", "review_rating"]).copy()

In [ ]:
# Converter 'review_rating' para numérico
test_set_clean["review_rating"] = pd.to_numeric(
    test_set_clean["review_rating"], errors="coerce"
)

In [ ]:
# Checar se há NaN após a conversão e remover essas linhas
test_set_clean = test_set_clean.dropna(subset=["review_rating"])

In [ ]:
# Função para categorizar as reviews
def binary_rating(rating):
    return "good" if rating >= 4.0 else "bad"

In [ ]:
# Limpar review_comment
test_set_clean["review_comment"] = test_set_clean["review_comment"].apply(
    preprocess_text
)

In [ ]:
# Aplicar 'binary_rating' para converter 'review_rating' em 'good' ou 'bad'
test_set_clean["review_rating"] = test_set_clean["review_rating"].apply(binary_rating)

In [ ]:
# Ter certeza que o processamento foi feito corretamente
test_set_clean.head()

In [ ]:
# Preparar as features e labels do modelo limpo, para teste
X_test_clean = test_set_clean["review_comment"]
Y_test_clean = test_set_clean["review_rating"]

In [ ]:
# Fazer previsões no conjunto de teste limpo
Y_test_pred_clean = pipeline_upsampled_binary.predict(X_test_clean)

In [ ]:
# Calcular a classificação do relatório para o conjunto de teste limpo
report_test_clean = classification_report(
    Y_test_clean, Y_test_pred_clean, output_dict=True
)

In [ ]:
# Calcular a matriz de confusão para o conjunto de teste limpo
conf_matrix_test_clean = confusion_matrix(Y_test_clean, Y_test_pred_clean)

In [ ]:
# Converter a classificação do relatório em um DataFrame
report_test_clean_df = pd.DataFrame(report_test_clean).transpose()

In [ ]:
# Plotar a matriz de confusão do conjunto de teste limpo em heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_test_clean,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["bad", "good"],
    yticklabels=["bad", "good"],
)
plt.title("Matriz de Confusão - Conjunto de Teste Limpo")
plt.xlabel("Previsão")
plt.ylabel("Real")
plt.show()

In [ ]:
print(report_test_clean_df)